In [1]:
%load_ext watermark
%watermark

2018-08-13T14:26:55-05:00

CPython 3.6.5
IPython 6.4.0

compiler   : MSC v.1900 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [6]:
!python -m cpppo.server.enip.client --udp --broadcast --list-identity -a 255.255.255.255

List Identity  0 from ('192.168.1.140', 44818): {
    "count": 1,
    "item[0].type_id": 12,
    "item[0].length": 46,
    "item[0].identity_object.version": 1,
    "item[0].identity_object.sin_family": 2,
    "item[0].identity_object.sin_port": 44818,
    "item[0].identity_object.sin_addr": "192.168.1.140",
    "item[0].identity_object.vendor_id": 1,
    "item[0].identity_object.device_type": 12,
    "item[0].identity_object.product_code": 200,
    "item[0].identity_object.product_revision": 2570,
    "item[0].identity_object.status_word": 48,
    "item[0].identity_object.serial_number": 12174641,
    "item[0].identity_object.product_name": "1756-EN2TR/C",
    "item[0].identity_object.state": 3
}


In [5]:
from cpppo.server.enip.get_attribute import proxy_simple
product_name, = proxy_simple( "192.168.1.140" ).read( [('@1/0/2','SINT')] )
product_name

[1, 0]

In [6]:
from cpppo.server.enip.get_attribute import proxy_simple
product_name, = proxy_simple( "192.168.1.140" ).read( [('@1/1/2','SINT')] )
product_name

[12, 0]

In [7]:
from cpppo.server.enip.get_attribute import proxy_simple
product_name, = proxy_simple( "192.168.1.140" ).read( [('@1/1/7','SSTRING')] )
product_name

['1756-EN2TR/C']

In [8]:
from __future__ import print_function
from cpppo.server.enip import client
import time

host = "192.168.1.140"
tags = [ "scada[0-10]", "scada[1]=99", "scada[0-10]" ]

with client.connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(
            operations=client.parse_operations( tags ), depth=2 ):
        print( "%s: %20s: %s" % ( time.ctime(), descr, value ))

Mon Aug 13 14:27:20 2018: Single Read  Tag  scada[0-10]: None
Mon Aug 13 14:27:20 2018: Single Write Tag  scada[1-1]: None
Mon Aug 13 14:27:20 2018: Single Read  Tag  scada[0-10]: None


In [9]:
from __future__ import print_function
from cpppo.server.enip import client
import time

host = "192.168.1.140"
tags = [ "scada[0-10]", "scada[1]=99", "scada[0-10]" ]

with client.connector( host=host ) as conn:
    req1 = conn.write( "scada[1-3]", data=[111,222,333] )
    req2 = conn.read( "scada[2]" )
    rpy1 = next( conn )
    rpy2 = next( conn )

In [10]:
from __future__ import print_function

import time

from cpppo.server.enip.client import connector
from cpppo.server.enip.get_attribute import attribute_operations

host = "192.168.1.140" # Your MicroLogix IP address
send_path = ''
route_path = False
attributes = [ '@1/1/1', '@1/1/7' ]
timeout = 1.0
depth = 1
multiple = 0

with connector( host=host ) as connection:
   operations = attribute_operations( attributes, send_path=send_path, route_path=route_path )
   for idx,dsc,op,rpy,sts,val in connection.pipeline(
           operations=operations, depth=depth, multiple=multiple, timeout=timeout ):
       print( "%s: %3d: (%-8s) %s == %s" % ( time.ctime(), idx, sts if sts else "OK", dsc, val ))

Mon Aug 13 14:27:21 2018:   0: (OK      ) Single G_A_S      @0x0001/1/1 == [1, 0]
Mon Aug 13 14:27:21 2018:   1: (OK      ) Single G_A_S      @0x0001/1/7 == [12, 49, 55, 53, 54, 45, 69, 78, 50, 84, 82, 47, 67]


In [13]:
from cpppo.server.enip.get_attribute import proxy
via = proxy( "192.168.1.140" )

with via:
    vendor, product_name = via.read( [('@1/1/1','INT'), ('@1/1/7','SSTRING')] )
    
print(vendor,product_name)

[1] ['1756-L71/B LOGIX5571']


In [8]:
from cpppo.server.enip.client import connector
import time

host = "192.168.1.140"
tags = [ "etiqueta_prueba", "etiqueta"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
        print( "{} | {} | {}".format( time.ctime(), descr, value ))
    conn.close()

Tue Aug 14 11:03:44 2018 | Single Read  Tag  etiqueta_prueba | [12345]
Tue Aug 14 11:03:44 2018 | Single Read  Tag  etiqueta | None


In [60]:
with connector(host=HOST) as conn:
    re = conn.read( "etiqueta_prueba")

In [66]:
print(re)
print(re['path']['segment'][0]['symbolic'])
re['path']['segment'][0]['symbolic']="etiqueta modificada"
print(re)
conn.close()
print(re)

{'path': {'segment': [{'symbolic': 'etiqueta modificada'}]}, 'read_frag': {'elements': 1, 'offset': 0}, 'service': 82, 'input': bytearray(b'R\t\x91\x0fetiqueta_prueba\x00\x01\x00\x00\x00\x00\x00')}
etiqueta modificada
{'path': {'segment': [{'symbolic': 'etiqueta modificada'}]}, 'read_frag': {'elements': 1, 'offset': 0}, 'service': 82, 'input': bytearray(b'R\t\x91\x0fetiqueta_prueba\x00\x01\x00\x00\x00\x00\x00')}
{'path': {'segment': [{'symbolic': 'etiqueta modificada'}]}, 'read_frag': {'elements': 1, 'offset': 0}, 'service': 82, 'input': bytearray(b'R\t\x91\x0fetiqueta_prueba\x00\x01\x00\x00\x00\x00\x00')}


In [32]:
from cpppo.server.enip import client
with connector( host=host ) as conn:
    conn.write("etiqueta", [123])

['path.segment[0].symbolic', 'write_frag.elements', 'write_frag.offset', 'write_frag.data', 'write_frag.type', 'service', 'input']


In [66]:
from cpppo.server.enip.client import connector
import time

host = "192.168.1.140"
tags = [ "etiqueta_prueba","etiqueta=(REAL)"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
        print( "{} | {} | {}".format( time.ctime(), descr, value ))
        print("{} \n | {} \n | {} \n | {} \n | {} \n | {}".format(index,descr,op,reply,status,value))
    
    req = conn.write("etiqueta_prueba", [123])
    assert conn.readable (timeout = 1.0), "Falló la recepción de la respuesta" 
    rpy = next (conn)
    conn.unconnected_send(req)
    for k,v in rpy.items():
        print("{} ======= {}".format(k,v))

Tue Aug 14 14:45:23 2018 | Single Read  Tag  etiqueta_prueba | [12345]
0 
 | Single Read  Tag  etiqueta_prueba 
 | {'path': {'segment': [{'symbolic': 'etiqueta_prueba'}]}, 'read_tag': {'elements': 1}, 'service': 76, 'input': bytearray(b'L\t\x91\x0fetiqueta_prueba\x00\x01\x00')} 
 | {'input': array('B', [204, 0, 0, 0, 196, 0, 57, 48, 0, 0]), 'service': 204, 'status': 0, 'status_ext': {'size': 0}, 'read_tag': {'type': 196, 'data': [12345]}} 
 | 0 
 | [12345]
Tue Aug 14 14:45:23 2018 | Single Write Tag  etiqueta | None
1 
 | Single Write Tag  etiqueta 
 | {'path': {'segment': [{'symbolic': 'etiqueta'}]}, 'write_tag': {'elements': 0, 'data': [], 'type': 202}, 'service': 77, 'input': bytearray(b'M\x05\x91\x08etiqueta\xca\x00\x00\x00')} 
 | {'input': array('B', [205, 0, 4, 1, 0, 0]), 'service': 205, 'status': 4, 'status_ext': {'size': 1, 'data': [0]}, 'write_tag': True} 
 | (4, [0]) 
 | None
peer ======= ('192.168.1.140', 44818)
enip.command ======= 111
enip.length ======= 22
enip.session_ha

In [65]:
host = "192.168.1.140"
tags = [ "etiqueta_prueba","etiqueta"]

with connector( host=host ) as conn:
    for index,descr,op,reply,status,value in conn.pipeline(operations=client.parse_operations( tags ), depth=2 ):
        print( "{} | {} | {}".format( time.ctime(), descr, value ))

Tue Aug 14 14:43:32 2018 | Single Read  Tag  etiqueta_prueba | [12345]
Tue Aug 14 14:43:32 2018 | Single Read  Tag  etiqueta | None
